In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
import numpy as np

In [5]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data.astype(np.float32)
y = iris.target.astype(np.int64)
N = y.size
y2 = np.zeros(3 * N).reshape(N, 3).astype(np.float32)

for i in range(N):
    y2[i, y[i]] = 1.0
    
index = np.arange(N)

np.save('train-x', X[index[index % 2 != 0]])
np.save('train-y', y2[index[index % 2 != 0]])
np.save('test-x', X[index[index % 2 == 0]])
np.save('test-y', y[index[index % 2 == 0]])

np.save('train-y0', y[index[index % 2 != 0]])

In [6]:
train_x = torch.from_numpy(np.load('train-x.npy'))
train_y = torch.from_numpy(np.load('train-y.npy'))
test_x = torch.from_numpy(np.load('test-x.npy'))
test_y = torch.from_numpy(np.load('test-y.npy'))

In [11]:
class MyIris(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(4, 6)
        self.l2 = nn.Linear(6, 3)
    def forward(self, x):
        h1 = torch.sigmoid(self.l1(x))
        h2 = self.l2(h1)
        return h2

In [12]:
model = MyIris()
optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.MSELoss()

In [13]:
for i in range(2000):
    output = model(train_x)
    loss = criterion(output, train_y)
    print(i, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 0.2656887173652649
1 0.25762370228767395
2 0.25310102105140686
3 0.2502143085002899
4 0.24810831248760223
5 0.24638862907886505
6 0.24486981332302094
7 0.24346330761909485
8 0.2421264797449112
9 0.24083910882472992
10 0.23959138989448547
11 0.23837874829769135
12 0.23719926178455353
13 0.23605190217494965
14 0.23493613302707672
15 0.23385144770145416
16 0.23279738426208496
17 0.2317732274532318
18 0.23077809810638428
19 0.2298109382390976
20 0.2288707196712494
21 0.22795610129833221
22 0.22706575691699982
23 0.22619827091693878
24 0.22535225749015808
25 0.22452622652053833
26 0.22371885180473328
27 0.22292867302894592
28 0.22215434908866882
29 0.22139456868171692
30 0.22064809501171112
31 0.21991373598575592
32 0.2191902995109558
33 0.21847684681415558
34 0.21777230501174927
35 0.21707573533058167
36 0.2163863182067871
37 0.21570317447185516
38 0.21502560377120972
39 0.21435292065143585
40 0.21368445456027985
41 0.21301960945129395
42 0.21235787868499756
43 0.2116987109184265
44 0.21

In [14]:
torch.save(model.state_dict(), 'my_iris.model')

In [15]:
model.load_state_dict(torch.load('my_iris.model'))

<All keys matched successfully>

In [16]:
model.eval()
with torch.no_grad():
    output1 = model(test_x)
    ans = torch.argmax(output1, 1)
    print((test_y == ans).sum().float() / len(ans))

tensor(0.9600)


## ミニバッチ

In [18]:
n = 25
bs = 75

for i in range(1000):
    idx = np.random.permutation(n)
    for j in range(0, n, bs):
        xtm = train_x[idx[j:(j+bs) if (j+bs) < n else n]]
        ytm = train_y[idx[j:(j+bs) if (j+bs) < n else n]]
        output = model(xtm)
        loss = criterion(output, ytm)
        print(i, j, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

0 0 0.0020231036469340324
1 0 0.0019130510045215487
2 0 0.0018607672536745667
3 0 0.0018300364026799798
4 0 0.0018075555562973022
5 0 0.0017883721739053726
6 0 0.0017706219805404544
7 0 0.0017535906517878175
8 0 0.0017369987908750772
9 0 0.001720728469081223
10 0 0.0017047401051968336
11 0 0.0016890044789761305
12 0 0.001673509250395
13 0 0.0016582489479333162
14 0 0.0016432196134701371
15 0 0.0016284104203805327
16 0 0.0016138209030032158
17 0 0.0015994476852938533
18 0 0.001585285528562963
19 0 0.0015713270986452699
20 0 0.0015575751895084977
21 0 0.001544020022265613
22 0 0.001530659501440823
23 0 0.0015174922300502658
24 0 0.001504512270912528
25 0 0.0014917192747816443
26 0 0.0014791075373068452
27 0 0.001466673333197832
28 0 0.0014544138684868813
29 0 0.0014423284446820617
30 0 0.0014304089127108455
31 0 0.00141865573823452
32 0 0.001407066942192614
33 0 0.0013956335606053472
34 0 0.0013843609485775232
35 0 0.0013732424704357982
36 0 0.0013622717233374715
37 0 0.00135145080275833

## 10. nn.Sequentialによるモデル設定と生成

#### データを変換していく各層に名前をつける必要がなくなる

In [19]:
model = nn.Sequential(
    nn.Linear(4, 6),
    nn.Sigmoid(),
    nn.Linear(6, 3))

## 11.nn.Modulelistによるモデル設定と生成

#### 各層をリストとして表していて、そのリストのインデックスによって層を参照する

In [20]:
class MyIris(nn.Module):
    def __init__(self):
        super().__init__()
        self.iris = nn.ModuleList(
        [nn.Linear(4, 6), nn.Sigmoid(), nn.Linear(6, 3)]
        )
    def forward(self, x):
        for i in range(len(self.iris)):
            x = self.iris[i](x)
        return x

## 12.GPUの利用

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')